In [1]:
# %%
import csv
import os
import cv2
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
from PIL import Image 
from pathlib import Path
import visualkeras
import mlflow
import mlflow.keras
from tensorflow.keras.applications import mobilenet_v3, efficientnet
import itertools # 確保 itertools 已匯入
from sklearn.utils.class_weight import compute_class_weight


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers, layers, models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint 
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

# from keras.optimizers import Adam
# from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import warnings 

warnings.filterwarnings('ignore')

In [2]:
# %%
# 1. 建立模型的函式 (已更新，支援兩階段訓練的凍結控制)
def build_model(model_name, learning_rate, dropout_rate, dense_units, trainable_layers, num_classes, freeze_base_model=False):
    """
    根據傳入的超參數建立並編譯指定的 Keras 模型。
    *** 使用 Functional API 以確保模型結構的穩健性 ***
    """
    input_shape = (224, 224, 3)
    
    # 定義模型輸入
    inputs = keras.Input(shape=input_shape)

    # 根據 model_name 選擇並建立基礎模型
    if model_name == 'MobileNetV3-Large':
        base_model = keras.applications.MobileNetV3Large(
            input_shape=input_shape,
            include_top=False,
            weights='imagenet',
            # 關鍵：將 base_model 的輸入與我們定義的 `inputs` 張量連接起來
            input_tensor=inputs 
        )
        print("✅ Base model loaded: MobileNetV3-Large")
    elif model_name == 'EfficientNet-B0':
        base_model = keras.applications.EfficientNetB0(
            input_shape=input_shape,
            include_top=False,
            weights='imagenet',
            # 關鍵：將 base_model 的輸入與我們定義的 `inputs` 張量連接起來
            input_tensor=inputs
        )
        print("✅ Base model loaded: EfficientNetB0")
    else:
        raise ValueError(f"不支援的模型名稱: {model_name}。")
        
    # --- 設定基底模型的可訓練性 ---
    if freeze_base_model:
        base_model.trainable = False
        print("🧊 Stage 1: Base model is FROZEN.")
    else:
        base_model.trainable = True
        if trainable_layers > 0 and trainable_layers < len(base_model.layers):
            for layer in base_model.layers[:-trainable_layers]:
                layer.trainable = False
            print(f"🔥 Stage 2: Fine-tuning. Last {trainable_layers} layers of base model are UNFROZEN.")
        else:
            print("🔥 Stage 2: Fine-tuning. All layers of base model are UNFROZEN.")

    # --- 使用 Functional API 串接模型 ---
    # 取得 base_model 的輸出
    x = base_model.output
    # 連接到後續的層
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    # 輸出層
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # 建立最終模型，明確指定輸入和輸出
    model = Model(inputs=inputs, outputs=outputs)
    
    # 編譯模型 (需要使用 Adam 的實例，而不是字串 'Adam')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [3]:
# 1. MLflow 視覺化與日誌記錄輔助函式 (維持不變)

def plot_and_log_history(history, filename="history_plots.png"):
    """
    繪製準確率和損失函數的歷史曲線，並將其儲存為圖片，最後記錄到 MLflow。
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 準確率圖
    ax1.plot(history.history['accuracy'], label='Train Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Val Accuracy')
    ax1.set_ylim(0, 1)
    ax1.set_title('Accuracy over epochs')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # 損失函數圖
    ax2.plot(history.history['loss'], label='Train Loss')
    ax2.plot(history.history['val_loss'], label='Val Loss')
    ax2.set_title('Loss over epochs')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    # 儲存圖片並關閉繪圖
    plt.tight_layout()
    plt.savefig(filename)
    plt.close(fig)  # 確保關閉正確的 figure
    
    # 將圖片記錄到 MLflow
    mlflow.log_artifact(filename)
    print(f"✅ '{filename}' has been logged to MLflow.")

def log_confusion_matrix(y_true, y_pred, class_labels, filename="confusion_matrix.png"):
    """ 繪製、儲存並記錄標準混淆矩陣 """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(18, 14)) # 增加圖片大小以容納更多類別
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels, cmap='Blues')
    plt.xlabel('Predicted Class')
    plt.ylabel('True Class')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()
    mlflow.log_artifact(filename)
    print(f"✅ '{filename}' has been logged to MLflow.")

def plot_and_log_sorted_confusion_bar(y_true, y_pred, class_labels, filename="sorted_correct_counts.png"):
    """ 計算、繪製並記錄一個長條圖，顯示每個類別的正確預測數量（由高到低排序）。 """
    cm = confusion_matrix(y_true, y_pred)
    correct_counts = np.diag(cm)
    
    sorted_idx = np.argsort(correct_counts)[::-1]
    sorted_counts = correct_counts[sorted_idx]
    sorted_labels = np.array(class_labels)[sorted_idx]

    plt.figure(figsize=(18, 8)) # 增加圖片寬度
    plt.bar(range(len(sorted_counts)), sorted_counts, color='lightgreen')
    plt.xticks(range(len(sorted_labels)), sorted_labels, rotation=90)
    plt.xlabel('Class')
    plt.ylabel('Correctly Predicted Count')
    plt.title('Correctly Predicted Count per Class (sorted)')
    plt.tight_layout()
    
    plt.savefig(filename)
    plt.close()

    mlflow.log_artifact(filename)
    print(f"✅ '{filename}' has been logged to MLflow.")

def evaluate_and_log_all_reports(model, data_generator, class_labels):
    """
    評估模型並記錄分類報告、混淆矩陣及其他分析圖到 MLflow。
    """
    print("\n" + "="*20 + " Starting Full Evaluation for Logging " + "="*20)

    # 關鍵：重置 generator 以確保順序正確
    data_generator.reset()

    # 1. 模型預測
    # 使用 predict() 方法，並指定 steps，可以確保處理完所有樣本
    predictions = model.predict(data_generator, steps=len(data_generator), verbose=1)
    y_pred = np.argmax(predictions, axis=1)

    # 2. 取得真實標籤
    # 注意：generator 的 classes 屬性包含了所有樣本的真實標籤
    y_true = data_generator.classes

    # 確保長度一致，這在 generator batch_size 無法整除總樣本數時很重要
    if len(y_pred) != len(y_true):
        print(f"⚠️ y_pred length {len(y_pred)} vs y_true length {len(y_true)}. This is normal if batch_size doesn't divide total samples.")
        # `predict` 會處理完所有樣本，所以 y_true 也應該是完整的
        y_true = y_true[:len(y_pred)]

    # 3. 產生並記錄分類報告
    report = classification_report(y_true, y_pred, target_names=class_labels, digits=4, zero_division=0)
    print("\nClassification Report:\n", report)
    mlflow.log_text(report, "classification_report.txt")
    print("✅ 'classification_report.txt' has been logged to MLflow.")

    # 4. 記錄混淆矩陣
    log_confusion_matrix(y_true, y_pred, class_labels)

    # 5. 記錄排序後的正確預測數量長條圖
    plot_and_log_sorted_confusion_bar(y_true, y_pred, class_labels)
    
    print("="*20 + " Full Evaluation Logging Complete " + "="*20 + "\n")

### 兩種模型

In [4]:
# %%
# 2. 自動訓練與調參的主函式 (已更新為使用 preprocess_input)
def train_and_tune(train_path, validation_path, target_accuracy):
    """
    自動執行兩階段訓練和超參數調整。
    為每個模型動態創建使用其專屬 preprocess_input 的數據生成器。
    """
    # 定義超參數的搜尋空間
    param_space = {
        'model_name': [
            'MobileNetV3-Large', 
            'EfficientNet-B0'],
        'learning_rate': [0.001],
        'dropout_rate': [0.3],
        'dense_units': [256],
        'trainable_layers': [80]  # 可調整的層數
    }
    
    STAGE_1_EPOCHS = 20
    STAGE_2_EPOCHS = 50
    FINE_TUNE_LR_MULTIPLIER = 0.1
    
    keys, values = zip(*param_space.items())
    param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    best_val_accuracy = 0
    best_run_id = None

    mlflow.set_experiment("MobileNetV3 & EfficientNet-B0 2階段訓練  資料集2(60)")
    
    for i, params in enumerate(param_combinations):
        if best_val_accuracy >= target_accuracy:
            print(f"🎯 目標已達成 (Accuracy >= {target_accuracy:.2f})。停止搜尋。")
            break

        print("\n" + "=" * 70)
        print(f"🚀 Run {i+1}/{len(param_combinations)}: 嘗試參數組合: {params}")

        # --- 關鍵修改：根據模型名稱選擇預處理函式 ---
        if params['model_name'] == 'MobileNetV3-Large':
            preprocessing_function = mobilenet_v3.preprocess_input
            print("INFO: Using MobileNetV3 preprocess_input.")
        elif params['model_name'] == 'EfficientNet-B0':
            preprocessing_function = efficientnet.preprocess_input
            print("INFO: Using EfficientNet preprocess_input.")
        else:
            raise ValueError(f"未知的模型: {params['model_name']}")

        # --- 在迴圈內部創建數據生成器 ---
        train_datagen_aug = ImageDataGenerator(
            preprocessing_function=preprocessing_function,  # 使用指定的函式
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            brightness_range=[0.9, 1.1],
            fill_mode='nearest'
        )
        
        validation_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

        train_gen = train_datagen_aug.flow_from_directory(
            train_path,
            target_size=(224, 224),
            batch_size=32,
            class_mode='categorical',
            shuffle=True
        )

        val_gen = validation_datagen.flow_from_directory(
            validation_path,
            target_size=(224, 224),
            batch_size=32,
            class_mode='categorical',
            shuffle=False
        )

        # 從生成器獲取數據集信息
        num_classes = train_gen.num_classes
        class_labels = list(train_gen.class_indices.keys())

        # 儲存類別名稱到 classes.csv
        with open('classes_new.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for cls in class_labels:
                writer.writerow([cls])
        
        # 計算 class_weight
        class_indices = np.unique(train_gen.classes)
        class_weights_array = compute_class_weight(
            class_weight='balanced',
            classes=class_indices,
            y=train_gen.classes
        )
        class_weight = dict(zip(class_indices, class_weights_array))

        # --- 訓練流程（與之前相同） ---
        with mlflow.start_run(run_name=f"{params['model_name']}_run_{i+1}") as run:
            mlflow.log_params(params)

            callbacks = [
                EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-8)
            ]
            
            # STAGE 1: 訓練分類頭
            print("\n" + "-" * 20 + " STAGE 1: Feature Extraction " + "-" * 20)
            model = build_model(
                **params, 
                num_classes=num_classes, 
                freeze_base_model=True
                )
            
            history_stage1 = model.fit(
                train_gen, 
                epochs=STAGE_1_EPOCHS, 
                validation_data=val_gen, 
                callbacks=callbacks, 
                class_weight=class_weight, 
                verbose=1
                )
            
            model.save_weights(f"checkpoints/stage1/{i+1}stage1.weights.h5")

            # STAGE 2: 微調
            print("\n" + "-" * 20 + " STAGE 2: Fine-Tuning " + "-" * 20)

            fine_tune_lr = params['learning_rate'] * FINE_TUNE_LR_MULTIPLIER
            params['learning_rate'] = fine_tune_lr
            model = build_model(
                **params, 
                num_classes=num_classes, 
                freeze_base_model=False
                )
            
            model.load_weights(f"checkpoints/stage1/{i+1}stage1.weights.h5")
            
            callbacks = [
                EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-9),
                ModelCheckpoint("model_checkpoint.keras", monitor="val_loss", save_best_only=True)
            ]
            history_stage2 = model.fit(train_gen, epochs=STAGE_2_EPOCHS, validation_data=val_gen, class_weight=class_weight, callbacks=callbacks, verbose=1)
            
            # MLflow 記錄
            full_history = {}
            for key in history_stage1.history.keys():
                full_history[key] = history_stage1.history[key] + history_stage2.history[key]
            
            val_accuracy = max(history_stage2.history['val_accuracy'])
            mlflow.log_metric("best_val_accuracy", val_accuracy)
            
            plot_and_log_history(type('History', (), {'history': full_history})())
            evaluate_and_log_all_reports(model, val_gen, class_labels)
            mlflow.keras.log_model(model, "model")
            
            print(f"✔️ Run {run.info.run_id} 完成。最佳驗證準確率: {val_accuracy:.4f}")
            
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                best_run_id = run.info.run_id
                print(f"🎉 新的最佳模型! Model: {params['model_name']}, Run ID: {best_run_id}, Accuracy: {best_val_accuracy:.4f}")
                model.save(f'checkpoints/bestmodel/{i+1}best_tuned_model.keras')
                print(f"💾 最佳模型已更新並儲存為 '{i+1}best_tuned_model.h5'")

    print("=" * 70)
    print("🏁 自動調參完成。")
    if best_run_id:
        print(f"🏆 最終最佳模型的 Run ID 為: {best_run_id}")
        print(f"🏆 最終最佳驗證準確率為: {best_val_accuracy:.4f}")
    else:
        print("❌ 未能成功訓練任何模型。")

In [6]:
# %%
# 3. 執行主流程 (已更新)

# --- 定義路徑 ---
train_path = "dataset_full_en_aug2/train"
validation_path = "dataset_full_en_aug2/validation"

# --- 啟動自動訓練與調參 ---
if __name__ == '__main__':
    # 函式內部會處理數據生成器、類別數量和權重計算
    train_and_tune(
        train_path=train_path,
        validation_path=validation_path,
        target_accuracy=0.98
    )


🚀 Run 1/2: 嘗試參數組合: {'model_name': 'MobileNetV3-Large', 'learning_rate': 0.001, 'dropout_rate': 0.3, 'dense_units': 256, 'trainable_layers': 80}
INFO: Using MobileNetV3 preprocess_input.
Found 26307 images belonging to 60 classes.
Found 7514 images belonging to 60 classes.

-------------------- STAGE 1: Feature Extraction --------------------
✅ Base model loaded: MobileNetV3-Large
🧊 Stage 1: Base model is FROZEN.
Epoch 1/20
823/823 ━━━━━━━━━━━━━━━━━━━━ 1331s 2s/step - accuracy: 0.4438 - loss: 2.0003 - val_accuracy: 0.6801 - val_loss: 1.1039 - learning_rate: 0.0010
Epoch 2/20
823/823 ━━━━━━━━━━━━━━━━━━━━ 1169s 1s/step - accuracy: 0.6121 - loss: 1.2995 - val_accuracy: 0.7072 - val_loss: 0.9815 - learning_rate: 0.0010
Epoch 3/20
823/823 ━━━━━━━━━━━━━━━━━━━━ 1100s 1s/step - accuracy: 0.6599 - loss: 1.1400 - val_accuracy: 0.7111 - val_loss: 0.9508 - learning_rate: 0.0010
Epoch 4/20
823/823 ━━━━━━━━━━━━━━━━━━━━ 1297s 2s/step - accuracy: 0.6911 - loss: 1.0359 - val_accuracy: 0.7372 - val_loss

2025/08/19 16:32:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/19 16:32:52 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


✅ 'sorted_correct_counts.png' has been logged to MLflow.
==================== Full Evaluation Logging Complete ====================



2025/08/19 16:33:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✔️ Run 2e5d33ac2f4d496db58c9e79ff17c965 完成。最佳驗證準確率: 0.8925
🎉 新的最佳模型! Model: MobileNetV3-Large, Run ID: 2e5d33ac2f4d496db58c9e79ff17c965, Accuracy: 0.8925
💾 最佳模型已更新並儲存為 '1best_tuned_model.h5'

🚀 Run 2/2: 嘗試參數組合: {'model_name': 'EfficientNet-B0', 'learning_rate': 0.001, 'dropout_rate': 0.3, 'dense_units': 256, 'trainable_layers': 80}
INFO: Using EfficientNet preprocess_input.
Found 26307 images belonging to 60 classes.
Found 7514 images belonging to 60 classes.

-------------------- STAGE 1: Feature Extraction --------------------


ValueError: Shape mismatch in layer #1 (named stem_conv)for weight stem_conv/kernel. Weight expects shape (3, 3, 1, 32). Received saved weight with shape (3, 3, 3, 32)